## About iPython Notebooks ##

iPython Notebooks are interactive coding environments embedded in a webpage. You will be using iPython notebooks in this class. Make sure you fill in any place that says `# BEGIN CODE HERE #END CODE HERE`. After writing your code, you can run the cell by either pressing "SHIFT"+"ENTER" or by clicking on "Run" (denoted by a play symbol). Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). 

 **What you need to remember:**

- Run your cells using SHIFT+ENTER (or "Run cell")
- Write code in the designated areas using Python 3 only
- Do not modify the code outside of the designated areas
- In some cases you will also need to explain the results. There will also be designated areas for that. 

Fill in your **NAME** and **AEM** below:

In [1]:
NAME = "Andreas Andreadis"
AEM = "2729"

---

# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [2]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [ ]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

In [3]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure and accuracy of your models.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three estimators/classifiers. Test both soft and hard voting and choose the best one.

In [4]:
# BEGIN CODE HERE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

cls1 = LogisticRegression(random_state=RANDOM_STATE)
cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE)
cls3 = GaussianNB()

# voting classifier
vcls = VotingClassifier(estimators=[('lr', cls1), ('dt', cls2), ('gauss', cls3)], voting='soft', n_jobs=-1)

scoring = {'accuracy' : make_scorer(accuracy_score), 'f1_score' : make_scorer(f1_score)}

# 10-fold cross-validation
cv_results = cross_validate(vcls, X, y, cv=10, n_jobs=-1, scoring = scoring)

avg_fmeasure = np.mean(cv_results['test_f1_score'])
#print("F1: ", avg_fmeasure)
avg_accuracy = np.mean(cv_results['test_accuracy'])
#print("Accuracy: ", avg_accuracy)

#END CODE HERE

In [5]:
print("Classifier:")
print(vcls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                 

### 1.2 Stacking ###
Create a stacking classifier which uses two estimators/classifiers. Try different classifiers for the combination of the initial classifiers. Report your results in the following cell.

In [7]:
# BEGIN CODE HERE
from sklearn.ensemble import StackingClassifier

# cls1 =  LogisticRegression(C = 1, max_iter = 100, random_state=RANDOM_STATE)
# cls2 =  LogisticRegression(C = 0.01, max_iter = 200, random_state=RANDOM_STATE)

# cls1 = DecisionTreeClassifier(max_depth = 5, random_state=RANDOM_STATE)
# cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE)

cls1 =  LogisticRegression(random_state=RANDOM_STATE)
cls2 =  DecisionTreeClassifier(random_state=RANDOM_STATE)

# level-0 classifiers
initial_estimators = [('cls1',cls1),('cls2',cls2)]

# stacking classifier
scls = StackingClassifier(estimators = initial_estimators, n_jobs=-1)

# 10-fold cv
cv_results = cross_validate(scls, X, y, cv=10, n_jobs=-1, scoring = scoring)

avg_fmeasure = np.mean(cv_results['test_f1_score'])
avg_accuracy = np.mean(cv_results['test_accuracy'])

# print("F1: ", avg_fmeasure)
# print("Accuracy: ", avg_accuracy)

#END CODE HERE

In [8]:
print("Classifier:")
print(scls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
StackingClassifier(cv=None,
                   estimators=[('cls1',
                                LogisticRegression(C=1.0, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                                                   n_jobs=None, penalty='l2',
                                                   random_state=42,
                                                   solver='lbfgs', tol=0.0001,
                                                   verbose=0,
                                                   warm_start=False)),
                               ('cls2',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                  

### 1.3 Report the results ###  
Report the results of your experiments in the following cell. How did you choose your initial classifiers? 

Το σημαντικό στο Stacking είναι οι level-0 classifiers να είναι διαφορετικοί, ώστε να κάνουν λάθος predictions σε διαφορετικά instances.

Χρησιμποιώντας ως meta-classifier τον default (LogisticRegression) έγιναν τα εξής πειράματα:

**1)** Mε ομογενές stacking χρησιμοποιώντας ως initial classifiers 2 Decision Trees και αλλάζοντας μόνο το max_depth έχουμε τα εξής αποτελέσματα:
    
  - **F1:** 0.776
  - **Accuracy:** 0.719
    
**2)** Ενώ με ετερογενές stacking χρησιμοποιώντας ως initial classifiers ένα Decision Tree και ένα Logistic Regression έχουμε τα εξής αποτελέσματα:
    
  - **F1:**  0.863
  - **Accuracy:**  0.838

**Συμπέρασμα:** Οι ετερογονείς ensembles, λόγω του diversity στο τρόπο κατασκευής και εκπαίδευσης των επιμέρους κατηγοριοποιητών είναι πιο αποδοτικοί.

Ωστόσο, ακόμα και αν χρησιμοποιήσουμε ομογενές ensemble μοντέλο μπορούμε να αυξήσουμε την απόδοση χρησιμοποιώντας **διαφορετικές παραμέτρους** στους initial classifiers. Για παράδειγμα, σε ένα μοντέλο με 2 Logistic Regression classifiers αν χρησιμοποιήσουμε στον έναν πιο μικρή τιμή για το C, δηλαδή φτιάξουμε ένα πιο γενικευμένο classifier ή αν αλλάξουμε τον αριθμό των iterations μπορούμε πάλι να κατασκευάσουμε ένα μοντέλο που συνολικά θα έχει πολύ καλή ακρίβεια. Για παράδειγμα, με τους παρακάτω initial classifiers:

>**LogisticRegression(C = 1, max_iter = 100, random_state=42)**

>**LogisticRegression(C = 0.01, max_iter = 200, random_state=42)**

Έχουμε τα εξής αποτελέσματα που είναι τα βέλτιστα μέχρι στιγμής:

- **F1:**  0.879
- **Accuracy:**  0.857

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one is produced with a different way from the ones discussed in the lecture for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1/accuracy score. The dictionaries should contain four different elements.  

In [9]:
# BEGIN CODE HERE
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import time

# bagging classifier
ens1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=42)
# random forest classifier
ens2 = RandomForestClassifier(random_state=RANDOM_STATE)
# adaboost classifier
ens3 = AdaBoostClassifier(n_estimators=50, random_state=RANDOM_STATE)
# a simple decision tree classifier 
tree = DecisionTreeClassifier(random_state=RANDOM_STATE)

# train-test splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# list of classifiers 
classifiers_list = [ens1, ens2, ens3, tree]
# list of classifiers' names
names_list = ['Bagging', 'Random Forest', 'AdaBoost', 'Decision Tree']

f_measures = dict()
accuracies = dict()
training_times = dict()

i = 0
# process each classifier one by one
for clf in classifiers_list:
    # fit train data
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    # make predictions
    y_pred = clf.predict(X_test)
    # round to 3 decimal points for simplicity
    training_time = round((end - start), 2)
    acc = round(accuracy_score(y_pred, y_test), 3)
    f1 = round(f1_score(y_pred, y_test), 3)
    # classifier name
    name = names_list[i]
    # update dictionaires
    f_measures.update({name: f1})
    accuracies.update({name: acc})
    training_times.update({name: training_time})
    i = i + 1

print('Times: ', training_times)
print('F1: ', f_measures)
print('Accuracy: ',accuracies)


#END CODE HERE

Times:  {'Bagging': 30.99, 'Random Forest': 5.04, 'AdaBoost': 19.12, 'Decision Tree': 4.2}
F1:  {'Bagging': 0.82, 'Random Forest': 0.857, 'AdaBoost': 0.831, 'Decision Tree': 0.748}
Accuracy:  {'Bagging': 0.793, 'Random Forest': 0.823, 'AdaBoost': 0.802, 'Decision Tree': 0.708}


In [10]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1:{}".format(name,score))
for name,score in accuracies.items():
    print("Classifier:{} -  Accuracy:{}".format(name,score))

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


**2.2** Describe your classifiers and your results.

**1) Bagging classifier**

Bagging ονομάζουμε τη διαδικασία κατα την οποία επιλέγουμε διαφορετικά samples με επανατοποθέτηση από τα αρχικά δεδομένα. O Bagging classifier είναι μια ensemble μέθοδος που εκπαιδεύει πολλούς classifiers (στη συγκεκριμένη περίπτωση DecisionTreeClassifier) κάθε έναν με διαφορετικά τυχαία δείγματα που δημιουργήθηκαν με τη διαδικασία του bagging. Με αυτόν τον τρόπο κάνει randomization στον τρόπο που κατασκευάζονται τα δέντρα. Στη συνέχεια, για να κάνει μια τελική πρόβλεψη ομαδοποιεί τις προβλέψεις από τους επιμέρους κατηγοριοποιητές (voting ή averaging). 

   - **Accuracy:** 0.782
   
   - **F1 Score:** 0.813

**2) Random Forest classifier**

Ο RandomForest βασίζεται πάλι στην μέθοδο με τα τυχαία samples του Bagging που αναφέρουμε παραπάνω. Η βασική διαφορά από τον BaggingClassifier είναι στην επιλογή των splits. O BaggingClassifier όταν καλείται να επιλέξει split point μπορεί να λάβει υπόψιν όλα τα features και να επιλέξει το optimal split. Από την άλλη ο RandomForest μπορεί να επιλέξει κάθε φορά μεταξύ m τυχαία επιλεγμένων features για να κάνει το split. Έτσι δημιουργούνται διαφορετικά υπό-μοντέλα (δέντρα με διαφορετικά splits) και με το συνδυασμό των predictions τους μπορεί να πετύχουμε καλύτερα αποτελέσματα.

   - **Accuracy:** 0.829
    
   - **F1 Score:** 0.863

**3) AdaBoost Classifier**

Στον 3ο classifier χρησιμοποιούμε τη μέθοδο του Boosting, κατά την οποία εκπαιδεύουμε μοντέλα διαδοχικά και κάθε μοντέλο προσπαθεί να διορθώσει τα λάθη του προηγούμενου. Αυτό πραγματοποιείται αναθέτοντας μεγαλύτερο βάρος στα instances που έγιναν missclassified στο προηγούμενο μοντέλο. Συγκεκριμένα χρησιμοποιούμε τον Adaboost classifier με τις default παραμέτρους, δηλαδή με 50 estimators (Decision Trees με max_depth = 1).

   - **Accuracy:** 0.805
   
   - **F1 Score:** 0.836

Η απόδοση του **Decision Tree Classifier** είναι η εξής:

   - **Accuracy:** 0.739
   
   - **F1 Score:** 0.768

**Παρατηρηση:** Τα 3 ομογενή ensemble μοντέλα πετυχαίνουν σαφώς καλύτερα αποτελέσματα απο τον απλό Decision Tree Classifier. Συγκεκριμένα με τις default παραμέτρους ο RandomForest αποδίδει καλύτερα από όλους τόσο σε θέμα ακρίβειας όσο και στον χρόνο που χρειάζεται για την εκπαίδευση.

**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

Με **10** estimators που είναι το default για τον Bagging classifier:

   - **Training Time:** ~ 34 sec.
    
Με **100** estimators έχουμε:

   - **Training Time:** ~ 335 sec 
    
Βλεπουμε οτι με 100 estimators θέλουμε 10 φορές περισσότερο χρόνο για την εκπαίδευση από ότι με 10. Ωστόσο, αν είχαμε περισσότερη υπολογιστική δύναμη θα μπορούσαμε να εκπαιδεύσουμε **παράλληλα** τους επιμέρους classifiers και να συνδυάσουμε στο τέλος τα αποτελέσματα τους εξοικονομώντας έτσι χρόνο.

Στους boosting classifiers δε θα μπορούσαμε να εφαρμόσουμε αυτή τη λύση, καθώς κάθε classifier περιμένει τα αποτελέσματα του προηγούμενου και εκπαιδεύεται προσπαθώντας να εστιάσει στα σημεία που έγινε λάθος classification. Αυτή η **ακολουθιακή** διαδικασία δε μας επιτρέπει να εκπαιδεύσουμε παράλληλα τους επιμέρους classifiers. 

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve an accuracy over 83-84%?

In [11]:
# BEGIN CODE HERE
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression


cls1 = RandomForestClassifier(random_state=RANDOM_STATE)
cls2 = AdaBoostClassifier(n_estimators=50, random_state=RANDOM_STATE)
cls3 = LogisticRegression(random_state=RANDOM_STATE)

best_cls = VotingClassifier(estimators=[
     ('rf', cls1), ('adaBoost', cls2), ('log', cls3)], voting='soft', n_jobs = -1)

scoring = {'accuracy' : make_scorer(accuracy_score), 'f1_score' : make_scorer(f1_score)}

cv_results = cross_validate(best_cls, X, y, cv=10, n_jobs=-1, scoring = scoring)

best_accuracy = np.mean(cv_results['test_accuracy'])
print("Accuracy: ", best_accuracy)
best_fmeasure = np.mean(cv_results['test_f1_score'])
print("F1: ", best_fmeasure)

#END CODE HERE

Accuracy:  0.8424769860211387
F1:  0.8666828404382505


In [12]:
print("Classifier:")
print(best_cls)
print("F1-Score:{} & Accuracy:{}".format(best_fmeasure,best_accuracy))

Classifier:
VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0

**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

Για να υλοποιήσω τον τελικό classifier ακολούθησα την εξής διαδικασία:

Αρχικά, δοκίμασα έναν RandomForest classifier που πέτυχε τα καλύτερα αποτελέσματα στην προηγούμενη άσκηση. Με την πρώτη προσπάθεια πετύχαμε τα εξής αποτελέσματα με ακρίβεια 3 δεκαδικών ψηφίων:

   >**- Accuracy:** 0.809

   >**- F1 Score:** 0.847

Δοκίμασα να αλλάξω τις παραμέτρους του RandomForest, άλλα η ακρίβεια παρέμεινε στα ίδια επίπεδα. Για να πλησιάσουμε στα επιθυμητά αποτελέσματα (83-84% accuracy) δοκιμάζουμε να προσθέσουμε ένα ακόμα classifier που υλοποιείται με διαφορετικό τρόπο, ώστε να κάνει λάθη σε διαφορετικά σημεία. Έχοντας ήδη έναν classifier που υλοποιεί την στρατηγική του **Bagging**, θα προσθέσουμε μια μέθοδο **boosting**. Δημιουργούμε έναν **voting classifier** με τον Random Forest και τον Adaboost ως τους 2 επιμέρους clasifiers του μοντέλου. Πλέον έχουμε τα εξής αποτελέσματα:

   >**- Accuracy:**  0.811
    
   >**- F1 Score:**  0.848

Παρατηρούμε μια μικρή βελτίωση στο μοντέλο μας, αλλά είμαστε ακόμα μακρυά από τον στόχο μας. Ένας τρόπος να αυξήσουμε και άλλο την ακρίβεια του μοντέλου είναι να προσθέσουμε έναν ακόμα διαφορετικό classifier. Μέχρι στιγμής τόσο ο Random Forest όσο και ο Adaboost βασίζονται σε δεντρικές μεθόδους. Προσθέτουμε λοιπόν έναν classifier που βασίζεται σε γραμμική μέθοδο. Προσθέτουμε έναν **LogisticRegression** classifier στον voting classifier. Τα νέα αποτελέσματα:

   >**- Accuracy:**  0.839
    
   >**- F1 Score:**  0.864

Προσθέτοντας και 4ο classifier μπορούμε να αυξήσουμε ακόμα περισσότερο την απόδοση του μοντέλου, αλλά ταυτόχρονα αυξάνεται αρκετά και ο χρόνος εκπαίδευσης. Έχοντας φτάσει πλέον το επιθυμητό αποτέλεσμα δε θα επεκτείνουμε περαιτέρω το μοντέλο μας.

**Παρατήρηση:** όσον αφορά τις παραμέτρους του τελικού **voting classifier**, χρησιμοποιήσαμε την πολιτική του **'soft voting'** καθώς μας έδωδε καλύτερα αποτελέσματα.

**!!!Τα αποτελέσματα ήταν λίγο διαφορετικά όταν έκανα restart τον kernel. Ισως κάπου έπρεπε να ορίσω random state και το παρέλειψα!!!**

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [13]:
# BEGIN CODE HERE
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

test_set = pd.read_csv("test_set_noclass.csv")
test_set.head()

cls1 = RandomForestClassifier(random_state=RANDOM_STATE)
cls2 = AdaBoostClassifier(n_estimators=50, random_state=RANDOM_STATE)
cls3 = LogisticRegression(random_state=RANDOM_STATE,  max_iter=1000)

cls = VotingClassifier(estimators=[('rf', cls1),('adaBoost', cls2), ('log', cls3)], voting='soft', n_jobs=-1)

cls.fit(X,y)

predictions = cls.predict(test_set)

#END CODE HERE

In [14]:
print(cls)
print(predictions)

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
        

LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

Για τα predictions χρησιμοποιήθηκε ο classifier που περιγράφεται στο **3.1** με τον οποίον πετύχαμε ακρίβεια κοντά στο **84%** με 10-fold cross validation

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
final_test_set = pd.read_csv('test_set.csv')
ground_truth = final_test_set['CLASS']
print("Accuracy:{}".format(accuracy_score(predictions,ground_truth)))
print("F1-Score:{}".format(f1_score(predictions,ground_truth)))